<img src="https://github.com/mousastech/agentes_ia/blob/a6db91737186d6d21d7808bb9950b154376d1d69/img/headertools_aiagents.png?raw=true" width=100%>

# Usando Agentes de IA

Capacitación práctica en la plataforma Databricks con enfoque en funcionalidades de IA generativa.

## Objetivo del ejercicio

El objetivo de este laboratorio es implementar el siguiente caso de uso:

### Personalización del servicio con Agentes

Los LLM son excelentes para responder preguntas. Sin embargo, esto por sí solo no es suficiente para ofrecer valor a sus clientes.

Para poder proporcionar respuestas más complejas, se requiere información adicional específica del usuario, como su ID de contrato, el último correo electrónico que envió a su soporte o su informe de compra más reciente.

Los agentes están diseñados para superar este desafío. Son despliegues de IA más avanzados, compuestos por múltiples entidades (herramientas) especializadas en diferentes acciones (recuperar información o interactuar con sistemas externos).

En términos generales, usted crea y presenta un conjunto de funciones personalizadas a la IA. Luego, el LLM puede razonar sobre qué información debe recopilarse y qué herramientas utilizar para responder a las instrucciones que recibe.
<br><br>

<img src="https://github.com/mousastech/agentes_ia/blob/e4602f57c4a83b171c7c541e11244136cdd80816/img/llm-call.png?raw=true" width="100%">

## Preparación

Para realizar los ejercicios, necesitamos prender a un Computo.

Simplemente siga los pasos a continuación:
1. En la esquina superior derecha, haga clic en **Conectar**
2. Seleccione el tipo de Clúster **SQL Serverless Warehouse** o **Serverless**.


## Conjunto de datos de ejemplo

Ahora, accedamos a las reseñas de productos que subimos en la práctica de laboratorio anterior.

En esta práctica de laboratorio usaremos dos tablas:
- **Evaluaciones**: datos no estructurados con el contenido de las evaluaciones
- **Clientes**: datos estructurados como registro de clientes y consumo.

¡Ahora visualicemos estos datos!

### A. Preparación de datos

1. Crear o utilizar el catalogo `agents_ia`
2. Crear o utilizar el schema `atencion`
3. Crear el volumen `archivos`
4. Importar los archivos de la carpeta `data` para el Volumen creado

Código disponible en el notebook `⚙️ ./Setup`

## Usando el Unity Catalog Tools

El primer paso en la construcción de nuestro agente será entender cómo utilizar **Unity Catalog Tools**.

En la práctica de laboratorio anterior, creamos algunas funciones, como `revisar_evaluacion`, que nos permitió facilitar la invocación de nuestros modelos de IA generativa desde SQL. Sin embargo, nuestros LLM también pueden utilizar estas mismas funciones como herramientas. ¡Simplemente indique qué funciones puede utilizar el modelo!

Poder utilizar el mismo catálogo de herramientas en toda la plataforma nos simplifica enormemente la vida al promover la reutilización de estos activos. Esto puede ahorrar horas de remodelación y estandarizar estos conceptos.

¡Veamos cómo utilizar las herramientas en la práctica!

1. En el **menú principal** de la izquierda, haz clic en **`Playground`**
2. Haz clic en el **selector de modelo** y selecciona el modelo **`Meta Llama 3.3 70B Instruct`** (si aún no está seleccionado)
3. Hacer clic **Tools** y luego en **Add Tool** 
4. En **Hosted Function**, tipear `agents_ia.atencion.revisa_resena`
5. Agregue instrucciones a continuación:
    ```
    Revise la reseña a continuación:
 Compré una tableta y estoy muy descontento con la calidad de la batería. Dura muy poco y tarda mucho en cargarse.
    ```
6. Haga clic en el ícono **enviar**

## Consulta de datos de clientes

Las herramientas se pueden utilizar en diferentes escenarios, como:

- Consultar información en bases de datos
- Calcular indicadores complejos
- Generar un texto basado en la información disponible
- Interactuar con API y sistemas externos

Como ya hemos comentado, esto será muy importante para que podamos producir respuestas más personalizadas y precisas en nuestro agente.

En nuestro caso nos gustaría:
- Consultar datos del cliente
- Búsqueda de preguntas y respuestas en una base de conocimientos
- Proporciona recomendaciones de productos personalizadas basadas en tus descripciones.

¡Comencemos consultando los datos de los clientes!

### A. Seleccione la base de datos creada previamente

In [0]:
USE agents_ia.atencion;

### B. Crear la función

In [0]:
CREATE OR REPLACE FUNCTION consulta_cliente(
    id_cliente BIGINT
)
RETURNS TABLE (
    nome STRING,
    sobrenome STRING,
    num_pedidos INT
)
COMMENT 'Utilice esta función para consultar los datos de un cliente'
RETURN 
    SELECT 
        c.nome AS nome,
        c.sobrenome AS sobrenome,
        c.num_pedidos AS num_pedidos
    FROM 
        clientes AS c
    WHERE 
        c.id_cliente = consulta_cliente.id_cliente

### C. Probar la función

In [0]:
SELECT * FROM consulta_cliente(2)

### D. Probar la función como herramienta.

1. En el **menú principal** de la izquierda, haz clic en **`Playground`**
2. Haga clic en el **selector de modelo** y seleccione el modelo **`Meta Llama 3.3 70B Instruct`** (si aún no está seleccionado)
3. Haga clic en **Tools** y luego en **Add Tools**
4. En **Hosted function**, escriba `agents_ia.atencion.consulta_cliente` y `agents_ia.atencion.revisa_resena`
5. Agregue las instrucciones a continuación:<br>
 Generar una respuesta al cliente 1 que no está satisfecho con la calidad de la pantalla de su tablet. No olvides personalizar el mensaje con el nombre del cliente.
6. Haga clic en el ícono **enviar**

### E. Análisis de los resultados

Con el resultado del ejercicio anterior, siga estos pasos:

1. En la parte inferior de la respuesta, haga clic en **`View trace`**

2. En este panel, navegue entre las diferentes acciones ejecutadas a la izquierda.

De esta manera, podrá comprender el razonamiento del agente: qué acciones se ejecutaron, con qué parámetros y en qué orden. Además, si se produce un error de ejecución, esto nos servirá de base para comprender y corregir cualquier problema.

## Búsqueda de preguntas y respuestas en una base de conocimientos

Ahora, necesitamos preparar una función que nos permita aprovechar una base de conocimientos para guiar las respuestas de nuestro agente.

Para hacer esto, usaremos **Vector Search**. Este componente nos permite comparar las preguntas formuladas por nuestro cliente con las de la base de conocimiento y luego recuperar la respuesta correspondiente a la pregunta con mayor similitud. ¡Lo único que debemos hacer es indexar las preguntas frecuentes, que subimos anteriormente, en Vector Search!

¡Vamos!

### A. Habilitar o Change Data Feed na tabela `FAQ`

Esta configuración permite a Vector Search leer los datos ingresados, eliminados o modificados en las preguntas frecuentes de forma incremental.

In [0]:
ALTER TABLE faq SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

### B. Crear un índice en Búsqueda de vectores (Vector Search)

Esta etapa tarda como 5 min!

1. No **menu principal** à esquerda, clique em **`Catalog`**
2. Busque a sua **tabela** `agents_ia.atencion.faq`
3. Clique em `Create` e depois em `Vector search index`
4. Preencha o formulário:
    - **Nome:** faq_index
    - **Primary key:** id
    - **Endpoint**: selecione o endpoint desejado
    - **Columns to sync:** deixar em branco (sincroniza todas as colunas)
    - **Embedding source:** Compute embeddings (Vector Search gestiona la indexación/creación de embeddings)
    - **Embedding source column:** pergunta
    - **Embedding model:** databricks-gte-large-en
    - **Sync computed embeddings:** desabilitado
    - **Sync mode:** Triggered
5. Clique em `Create`
6. Aguarde a criação do índice finalizar

### C. Crear la función

In [0]:
CREATE OR REPLACE FUNCTION consulta_faq(pergunta STRING)
RETURNS TABLE(id LONG, pregunta STRING, respuesta STRING, search_score DOUBLE)
COMMENT 'Utilice esta función para consultar la base de conocimientos sobre tiempos de entrega, solicitudes de cambio o devolución, entre otras preguntas frecuentes sobre nuestro mercado.'
RETURN select * from vector_search(
  index => 'agents_ia.atencion.faq_index', 
  query => consulta_faq.pergunta,
  num_results => 1
)

In [0]:
CREATE OR REPLACE FUNCTION agents_ia.atencion.consulta_faq(pregunta STRING)
RETURNS STRING
COMMENT 'Utilice esta función para consultar la base de conocimientos sobre tiempos de entrega, solicitudes de cambio o devolución, entre otras preguntas frecuentes sobre nuestro mercado.'
RETURN (
  SELECT resposta 
  FROM vector_search(
    index => 'agents_ia.atencion.faq_index', 
    query => pregunta,
    num_results => 1
  )
  LIMIT 1
)

### D. Probar la función

In [0]:
SELECT consulta_faq('¿Cuál es el plazo de devolución?') AS STRING

In [0]:
SELECT consulta_faq('¿Cómo emitir un duplicado?')

### E. Pruebe la función como herramienta.

1. En el **menú principal** de la izquierda, haz clic en **`Playground`**
2. Haga clic en el **selector de modelo** y seleccione el modelo **`Meta Llama 3.3 70B Instruct`** (si aún no está seleccionado)
3. Haga clic en **Tools** y luego en **Add Tools**
4. En **Hosted Function**, escriba `agents_ia.atencion.consulta_faq`
5. Agregue la siguiente declaración:
 ```
 ¿Cuál es el plazo de devolución?
 ```
6. Haga clic en el ícono **enviar**

## Proporcionar recomendaciones de productos personalizadas basadas en sus descripciones.

Finalmente, también nos gustaría crear una herramienta para ayudar a nuestros clientes a encontrar productos que tengan descripciones similares. Esta herramienta ayudará a los clientes que no están satisfechos con un producto y buscan un cambio.

### A. Habilite Change Data Feed en la tabla `productos`

In [0]:
ALTER TABLE productos SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

### B. Crear un índice en el Vector Search

1. En el **menú principal** de la izquierda, haga clic en **`Catálogo`**
2. Busca tu **tabla** `agents_ia.atencion.productos`
3. Haga clic en `Create` y luego en `Vector search index`
4. Complete el formulario:
 - **Nombre:** productos_index
 - **Primary key:** id
 - **Endpoint**: seleccione el punto final deseado
 - **Columns to sync:** dejar en blanco (sincroniza todas las columnas)
 - **Fuente de incrustación:** Computar incrustaciones (Vector Search gestiona la indexación/creación de incrustaciones)
 - **Embedding source:** descripción
 - **Embedding model:** databricks-gte-large-en
 - **Sync computed embeddings:** deshabilitado
 - **Sync mode:** Activado
5. Haga clic en "Create".
6. Espere a que finalice la creación del índice.

### C. Crear la función

In [0]:
CREATE OR REPLACE FUNCTION busca_productos_similares(descripcion STRING)
RETURNS TABLE(id LONG, producto STRING, descripcion STRING, search_score DOUBLE)
COMMENT 'Esta función recibe una descripción del producto, que se utiliza para buscar productos similares.'
RETURN SELECT id, produto, descricao, search_score FROM (
  SELECT *, ROW_NUMBER() OVER (ORDER BY search_score DESC) AS rn
  FROM vector_search(
    index => 'agents_ia.atencion.productos_index',
    query => busca_productos_similares.descripcion,
    num_results => 10)
  WHERE search_score BETWEEN 0.003 AND 0.99
) WHERE rn <= 3

### D. Probar la función

In [0]:
SELECT * FROM busca_productos_similares('Los auriculares DEF son un dispositivo de audio diseñado para brindar una experiencia de sonido envolvente y de alta calidad. Con controladores de alta fidelidad y tecnología de cancelación de ruido, te permite perderte en la música o los detalles de un podcast sin distracciones. Además, su diseño ergonómico garantiza comodidad durante un uso prolongado.')

### E. Probar la función como herramienta

1. En el **menú principal** de la izquierda, haz clic en **`Playground`**
2. Haga clic en el **selector de modelo** y seleccione el modelo **`Meta Llama 3.3 70B Instruct`** (si aún no está seleccionado)
3. Haga clic en **Tools** y luego en **Add tools**
4. En **Hosted function**, escriba `agents_ia.atencion.busca_productos_similares`
5. Agregue la siguiente declaración:
 ```
 ¿Qué tabletas tienen buena calidad de pantalla?
 ```
6. Haga clic en el ícono **enviar**

## Probando a nuestro agente

1. En el **menú principal** de la izquierda, haz clic en **`Playground`**
2. Haga clic en el **selector de modelo** y seleccione el modelo **`Meta Llama 3.3 70B Instruct`** (si aún no está seleccionado)
3. Haga clic en **Tools** y luego en **Add tools**
4. En **Hosted Function**, escriba `agents_ia.atencion.*` para agregar todas las funciones creadas.
5. En **Evaluation** apagar _AI Judge_ y _Synthetic question generation_
5. En **System Prompt**, escriba: <br>
`Eres un asistente virtual para un comercio electrónico. Para responder preguntas, el cliente debe proporcionar su cédula. Si aún no tiene esta información, solicite cortésmente su cédula. Podrás resolver dudas sobre entrega, devoluciones de productos, estado de pedidos, entre otras. Si no sabe cómo responder la pregunta, diga que no lo sabe. No inventes ni especules sobre nada. Siempre que se le pregunte sobre procedimientos, consulte nuestra base de conocimientos.`
6. Escribe "¡Hola!"
7. Ingrese el número de cédula `000.000.000-01`
8. Escriba "Compré una tableta DEF, pero la calidad de la pantalla es muy mala".
9. Escriba "¿Podría recomendar productos similares?"
10. Escriba "¿Cómo solicito un cambio?"

# ¡Felicidades!

¡Has completado su primer laboratorio de **Agentes**!

¡Ahora ya sabes cómo utilizar Foundation Models, Playground y las herramientas de catálogo de Unity para crear prototipos de forma rápida y sencilla de agentes capaces de responder con precisión preguntas complejas!